In [13]:
!wget https://www.dropbox.com/scl/fi/x77x88yfhnsjrfgn20b15/titanic_test.csv?rlkey=xrk50r09fcys0qkz0soay4q49&st=4s9z7rtf&dl=0
!wget https://www.dropbox.com/scl/fi/2t3qxsju9knmad46ayv0e/titanic_train.csv?rlkey=t0tectfva29pdxfhqrzdfph8x&st=rkhxbm8u&dl=0

--2025-03-13 11:54:30--  https://www.dropbox.com/scl/fi/x77x88yfhnsjrfgn20b15/titanic_test.csv?rlkey=xrk50r09fcys0qkz0soay4q49
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbd277ab3299b248b1e080152b2.dl.dropboxusercontent.com/cd/0/inline/ClymwXfTX54qW033zapnE3OKvJwDtx6_0HEAHd6VRwAGQwe4CbDQ3SuqQt54rpxL9HNSbaOBCnfGuXUpFyhwACjR83MSCpkSfReRk3OQM22veAt7WycSLb9_pUZ2s7aIc-G1HF81aZzox2QCDwarKxOl/file# [following]
--2025-03-13 11:54:31--  https://ucbd277ab3299b248b1e080152b2.dl.dropboxusercontent.com/cd/0/inline/ClymwXfTX54qW033zapnE3OKvJwDtx6_0HEAHd6VRwAGQwe4CbDQ3SuqQt54rpxL9HNSbaOBCnfGuXUpFyhwACjR83MSCpkSfReRk3OQM22veAt7WycSLb9_pUZ2s7aIc-G1HF81aZzox2QCDwarKxOl/file
Resolving ucbd277ab3299b248b1e080152b2.dl.dropboxusercontent.com (ucbd277ab3299b248b1e080152b2.dl.dropboxusercontent.com)... 162.125.3.15, 26

In [14]:
import os

os.rename("titanic_train.csv?rlkey=t0tectfva29pdxfhqrzdfph8x", "train.csv")
os.rename("titanic_test.csv?rlkey=xrk50r09fcys0qkz0soay4q49", "test.csv")

In [15]:
import pandas as pd

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
df = pd.concat([train_data, test_data], axis=0)

# Adatok megfigyelése

In [16]:
# Látható, hogy az Age hiányos több sorban is
# Látható, hogy vannak kiugró értékek a Fair-ben, valószínüleg azért, mert több embernek fizette
# Látható, hogy a SibSp és Parch értékek is általánosságban alacsonyak,
# csak a felső 25%-ban jelennek meg nagyobb értékek, az alatt 0 vagy 1
# Egy Fare érték hiányzik, ezt valahogy meg kell oldani

pd.options.display.float_format = '{:.2f}'.format
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.00,891.00,1309.00,1046.00,1309.00,1309.00,1308.00
mean,655.00,0.38,2.29,29.88,0.50,0.39,33.30
std,378.02,0.49,0.84,14.41,1.04,0.87,51.76
min,1.00,0.00,1.00,0.17,0.00,0.00,0.00
25%,328.00,0.00,2.00,21.00,0.00,0.00,7.90
50%,655.00,0.00,3.00,28.00,0.00,0.00,14.45
75%,982.00,1.00,3.00,39.00,1.00,0.00,31.27
max,1309.00,1.00,3.00,80.00,8.00,9.00,512.33


In [17]:
# Látható, hogy az Age, Cabin és Embarked tartalmaznak NaN értékeket
# A Survived NaN értékei a Test adatnak tudhatóak be, az adatkezelés egyszerűsége végett kerültek egy dataframebe
df.isnull().sum()

,0
PassengerId,0
Survived,418
Pclass,0
Name,0
Sex,0
Age,263
SibSp,0
Parch,0
Ticket,0
Fare,1


In [18]:
# Cabin esetében sok különböző érték van --> nehéz prediktálni, hogy mi lehetett a helyes érték
# Ticketek értékei ismétlődnek, feltételezhetően a családok miatt, ebbe érdemes lehet belenézni
# Embarked értékek közül 2 hiányzik, meg kell oldani

df[["Cabin", "Ticket", "Embarked"]].describe()

,Cabin,Ticket,Embarked
count,295,1309,1307
unique,186,929,3
top,C23 C25 C27,CA. 2343,S
freq,6,11,914


In [19]:
# 7 tagú család pl. -- Ticket az előző táblázat top értéke

filtered_data = df[df['Ticket'] == '347082']
print(filtered_data[['Name', 'Parch', 'SibSp', 'Age']])

                                                  Name  Parch  SibSp   Age
13                         Andersson, Mr. Anders Johan      5      1 39.00
119                  Andersson, Miss. Ellis Anna Maria      2      4  2.00
541               Andersson, Miss. Ingeborg Constanzia      2      4  9.00
542                  Andersson, Miss. Sigrid Elisabeth      2      4 11.00
610  Andersson, Mrs. Anders Johan (Alfrida Konstant...      5      1 39.00
813                 Andersson, Miss. Ebba Iris Alfrida      2      4  6.00
850            Andersson, Master. Sigvard Harald Elias      2      4  4.00


# Adat Feldolgozás, Takarítás

## Fölösleges Oszlopok eldobása

In [20]:
# A Ticketet és PassangerId-t törlöljük, mert nem fogunk tudni hasznos adatot kinyerni belőlük
df = df.drop('Ticket', axis=1)
df = df.drop('PassengerId', axis=1)

# Nagyon sok féle értéket vesz fel, nehéz lehet pontosan meghatározni az ismeretlen értékeket
print(df["Cabin"].value_counts())
print()

# Látható, hogy a Cabin legtöbbször akkor ismert, ha 1. osztályon utaztak, de vannak kivételek
print(df[df["Cabin"].notnull()]["Pclass"].value_counts())

# Mivel a hajó lassan süllyedt el, ezért a Cabin tulajdonságból nem vonható le következtetés abban a tekintetben, hogy valamely kabinokból
# már nem lett volna idő kimenekülni. Illetve az is adott, hogy minél drágább osztályon utazik valaki, annál magasabb szinten lakott, ezt megadja a PClass
# Ezért úgy értékeltük, hogy ennek az oszlopnak nincs információ értéke, ezért eldobjuk
df = df.drop('Cabin', axis=1)

https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/

We have a ‘Cabin’ column not doing much, only 1st class passengers have cabins, the rest are ‘Unknown’. A cabin number looks like ‘C123’. The letter refers to the deck, and so we’re going to extract these just like the titles.

Turning cabin number into Deck

cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']

df['Deck']=df['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))

## Age NaN értékek megoldása

In [22]:
# Létrehozunk egy új oszlopot, amiben az utasok titulusai szerepelnek
title = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df['Title'] = title
print(title.value_counts())
print()

# Megfigyeltem (manuálisan, mert nem sok), hogy minden címmel rendelkező (Age nélküli!) ember esetén van
# legalább egy hasonló címmel rendelkező ember, akinek a korát fel lehet használni a pótláshoz
df[df['Title'] == "Jonkheer"]

Name
Mr          757
Miss        260
Mrs         197
Master       61
Rev           8
Dr            8
Col           4
Mlle          2
Major         2
Ms            2
Lady          1
Sir           1
Mme           1
Don           1
Capt          1
Countess      1
Jonkheer      1
Dona          1
Name: count, dtype: int64



,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
822,0.00,1,"Reuchlin, Jonkheer. John George",male,38.00,0,0,0.00,S,Jonkheer


In [23]:
# Age-ben NaN értékek kicserélése az ugyanolyan Title-el rendelkező utasok Age értékének mediánjával
for title in df['Title'].unique():
    median_age = df[df['Title'] == title]['Age'].median()
    df.loc[(df['Title'] == title) & (df['Age'].isnull()), 'Age'] = median_age

##Family attribútum hozzáadása

In [ ]:
# Ez utólag nem bizonyult hasznosnak a korrelációra alapozva
#train_df['Family'] = train_df['SibSp'] + train_df['Parch'] + 1
#test_df['Family'] = test_df['SibSp'] + test_df['Parch'] + 1

In [25]:
# több tényezősnek érzem azt, hogy ki mennyit fizet, mint szimplán az Embarked mivolta,
# de végülis ugyanazt az eredményt adja, mint lejjebb

median_fares_by_family = df.groupby(['Sex','SibSp','Parch','Embarked'])['Fare'].median()
median_fares_by_embarked = df.groupby('Embarked')['Fare'].median()
print(median_fares_by_family)
print()
print(median_fares_by_embarked)

Sex     SibSp  Parch  Embarked
female  0      0      C          63.11
                      Q           7.75
                      S          12.47
               1      C          59.40
                      S          24.50
                                  ... 
male    4      1      Q          29.12
                      S          39.69
               2      S          31.39
        5      2      S          46.90
        8      2      S          69.55
Name: Fare, Length: 75, dtype: float64

Embarked
C   28.52
Q    7.75
S   13.00
Name: Fare, dtype: float64


## Embarked NaN értékek megoldása

In [26]:
# Megfigyeltem, hogy a felszállás különböző helyein az átlagok és mediánok is jelentősen különböznek,
# ezt használom ki a Embarked érték meghatározására
print(df[df["Embarked"] == "Q"]["Fare"].describe())

# Feltöltöm az embarked üres értékeit medián fare alapján legvalószínűbb kikötővel
for index, row in df[df['Embarked'].isnull()].iterrows():
    distances = abs(median_fares_by_embarked - row['Fare'])
    df.loc[index, 'Embarked'] = distances.idxmin()

count   123.00
mean     12.41
std      13.62
min       6.75
25%       7.75
50%       7.75
75%      10.20
max      90.00
Name: Fare, dtype: float64


## Fare NaN értékek megoldása

In [27]:
# Feltöltöm a Fare üres értékeit az adott kikötőhöz tarzozó medián Fare értékkel
df['Fare'] = df['Fare'].fillna(df.groupby('Embarked')['Fare'].transform('median'))

# Hasonló megközelítés az Embarked-hoz, csak fordítva, Embarked alapján adom meg a Fair-t
#for index, row in df[df['Fare'].isnull()].iterrows():
#    df.loc[index, 'Fare'] = median_fares[row['Embarked']]
#df[df["PassengerId"] == 1044]

# Minden értéket feltöltöttünk, a 418 NaN a Survived oszlopban a tanító adat miatt van
df.isnull().sum()

,0
Survived,418
Pclass,0
Name,0
Sex,0
Age,0
SibSp,0
Parch,0
Fare,0
Embarked,0
Title,0


Ezzel minden NaN attribútumot feltöltöttünk

# Sex one-hot enkódolás

In [28]:
# Nemet és felszállási helyet one-hot enkódolással oldom meg, hogy könnyebben felhasználható legyen
dummies = pd.get_dummies(df["Sex"], prefix='Sex')
df = pd.concat([df, dummies], axis=1)

## Embarked one-hot enkódolás

In [29]:
dummies = pd.get_dummies(df["Embarked"], prefix='Embarked')
df = pd.concat([df, dummies], axis=1)

## Végső DataFrame-k

In [30]:
train_df = df[df['Survived'].notna()]
test_df = df[df['Survived'].isna()]

In [31]:
x_cols = ['Pclass', 'Age',  'Fare','SibSp', 'Parch', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S']
y_cols = ['Survived']

In [32]:
# Látható, hogy a túlélés jobban korrelál az utas jegyének osztályával, nemével, a jegyének árával, illetve azzal, hogy C vagy S-nél szállt fel.
# Látható, hogy nőknek és gazdagabbaknak nagyobb esélye volt túlélni
train_df[x_cols + y_cols].corr()['Survived']

,Survived
Pclass,-0.34
Age,-0.07
Fare,0.26
SibSp,-0.04
Parch,0.08
Sex_female,0.54
Sex_male,-0.54
Embarked_C,0.17
Embarked_Q,0.00
Embarked_S,-0.16


In [33]:
# Q értéke azért nem számít annyira valószínűleg, mert Q-nál kevesebb gazdagabb utas szállt fel, ami könnyítené ennek eldöntését
train_df[x_cols + y_cols].corr()

,Pclass,Age,Fare,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Survived
Pclass,1.00,-0.36,-0.55,0.08,0.02,-0.13,0.13,-0.25,0.22,0.08,-0.34
Age,-0.36,1.00,0.10,-0.26,-0.18,-0.09,0.09,0.05,-0.06,-0.01,-0.07
Fare,-0.55,0.10,1.00,0.16,0.22,0.18,-0.18,0.27,-0.12,-0.17,0.26
SibSp,0.08,-0.26,0.16,1.00,0.41,0.11,-0.11,-0.06,-0.03,0.07,-0.04
Parch,0.02,-0.18,0.22,0.41,1.00,0.25,-0.25,-0.01,-0.08,0.06,0.08
Sex_female,-0.13,-0.09,0.18,0.11,0.25,1.00,-1.00,0.09,0.07,-0.13,0.54
Sex_male,0.13,0.09,-0.18,-0.11,-0.25,-1.00,1.00,-0.09,-0.07,0.13,-0.54
Embarked_C,-0.25,0.05,0.27,-0.06,-0.01,0.09,-0.09,1.00,-0.15,-0.78,0.17
Embarked_Q,0.22,-0.06,-0.12,-0.03,-0.08,0.07,-0.07,-0.15,1.00,-0.50,0.00
Embarked_S,0.08,-0.01,-0.17,0.07,0.06,-0.13,0.13,-0.78,-0.50,1.00,-0.16
